In [1]:
#!pip install qiskit
#!pip install qiskit-aer
#!pip install qiskit_ibm_runtime
#!pip install MatPlotLib
#!pip install pylatexenc
#!pip freeze

In [1]:
#common imports
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.result import QuasiDistribution #for sampler results

In [2]:


# Create a new circuit with two qubits
def my_circuit():
  qc = QuantumCircuit(2)

  # Add a Hadamard gate to qubit 0
  qc.h(0)

  # Perform a controlled-X gate on qubit 1, controlled by qubit 0
  qc.cx(0, 1)
  return qc

# Return a drawing of the circuit using MatPlotLib ("mpl"). This is the
# last line of the cell, so the drawing appears in the cell output.
# Remove the "mpl" argument to get a text drawing.
qc=my_circuit()
qc.draw("mpl")

observables_labels = ["ZI","IZ"]
#observables_labels = ["IZ"]
observables = [SparsePauliOp(label) for label in observables_labels]

# Construct the Estimator instance we want to use.


#estimator.options.resilience_level = 1
#estimator.options.default_shots = 5000


observables = [
    observable.apply_layout(qc.layout) for observable in observables
]


In [3]:
#Default Sampler and Estimator
from qiskit.primitives import StatevectorSampler as DefSamplerV2
from qiskit.primitives import StatevectorEstimator as DefEstimatorV2

print(f"\n### Estimator Usage: ###")

estimator=DefEstimatorV2()
precision=None #or between 0.0 and 1.0

qc=my_circuit()
# Estimator Usage
# One pub, with one circuit to run against five different observables.
job = estimator.run([(qc, observables)], precision=precision)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]
print(f"The primitive-job finished with result {pub_result}")
print(f"  > Observables: {observables_labels}")
print(f"  > Expectation value: {job_result[0].data.evs}")
print(f"  > Standard deviation: {job_result[0].data.stds}")
print(f"  > Metadata: {job_result[0].metadata}")

#Sampler usage
sampler=DefSamplerV2()
num_shots=1024
print(f"\n### Sampler Usage: ###")

qc2 =my_circuit()
qc2.measure_all() ## add a measure all layer
# One pub, with one circuit to run against five different observables.
job = sampler.run([(qc2)], shots=num_shots)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]

print(f"The primitive-job finished with result {pub_result}")

counts = (
    pub_result.data.meas.get_int_counts()
)
dist = QuasiDistribution(
    {meas: count / num_shots for meas, count in counts.items()}, shots=num_shots
)

print(counts)
print(dist)


### Estimator Usage: ###
The primitive-job finished with result PubResult(data=DataBin<2>(evs=[0. 0.], stds=[0. 0.]), metadata={'precision': 0.0})
  > Observables: ['ZI', 'IZ']
  > Expectation value: [0. 0.]
  > Standard deviation: [0. 0.]
  > Metadata: {'precision': 0.0}

### Sampler Usage: ###
The primitive-job finished with result PubResult(data=DataBin<>(meas=BitArray(<shape=(), num_shots=1024, num_bits=2>)), metadata={'shots': 1024})
{3: 543, 0: 481}
{3: 0.5302734375, 0: 0.4697265625}


In [4]:
#Default ibm noisy simulation
from qiskit_ibm_runtime.fake_provider import FakeHanoi
from qiskit_ibm_runtime import EstimatorV2 as IbmEstimatorV2
from qiskit_ibm_runtime import SamplerV2 as IbmSamplerV2

fake_backend=FakeHanoi()

print(f"\n### Estimator Usage: ###")
estimator=IbmEstimatorV2(fake_backend)
#precision=0.1 #None #or between 0.0 and 1.0
precision=0.1 #None #or between 0.0 and 1.0
qc=my_circuit()
# Estimator Usage
# One pub, with one circuit to run against five different observables.
job = estimator.run([(qc, observables)], precision=precision)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]
print(f"The primitive-job finished with result {pub_result}")
print(f"  > Observables: {observables_labels}")
print(f"  > Expectation value: {job_result[0].data.evs}")
print(f"  > Standard deviation: {job_result[0].data.stds}")
print(f"  > Metadata: {job_result[0].metadata}")

#Sampler usage
sampler=IbmSamplerV2(fake_backend)
num_shots=1024
print(f"\n### Sampler Usage: ###")

qc2 =my_circuit()
qc2.measure_all() ## add a measure all layer
# One pub, with one circuit to run against five different observables.
job = sampler.run([(qc2)], shots=num_shots)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]

print(f"The primitive-job finished with result {pub_result}")

counts = (
    pub_result.data.meas.get_int_counts()
)
dist = QuasiDistribution(
    {meas: count / num_shots for meas, count in counts.items()}, shots=num_shots
)

print(counts)
print(dist)


### Estimator Usage: ###
The primitive-job finished with result PubResult(data=DataBin<2>(evs=[0.   0.02], stds=[0.1     0.09998]), metadata={'target_precision': 0.1})
  > Observables: ['ZI', 'IZ']
  > Expectation value: [0.   0.02]
  > Standard deviation: [0.1     0.09998]
  > Metadata: {'target_precision': 0.1}

### Sampler Usage: ###
The primitive-job finished with result PubResult(data=DataBin<>(meas=BitArray(<shape=(), num_shots=1024, num_bits=2>)))
{0: 509, 3: 494, 1: 4, 2: 17}
{0: 0.4970703125, 3: 0.482421875, 1: 0.00390625, 2: 0.0166015625}


In [5]:
#Aer simulation no noise
from qiskit_aer.primitives import EstimatorV2 as AerEstimatorV2
from qiskit_aer.primitives import SamplerV2 as AerSamplerV2

print(f"\n### Estimator Usage: ###")

estimator=AerEstimatorV2()
precision=0.1 #None #or between 0.0 and 1.0
qc=my_circuit()
# Estimator Usage
# One pub, with one circuit to run against five different observables.
job = estimator.run([(qc, observables)], precision=precision)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]
print(f"The primitive-job finished with result {pub_result}")
print(f"  > Observables: {observables_labels}")
print(f"  > Expectation value: {job_result[0].data.evs}")
print(f"  > Standard deviation: {job_result[0].data.stds}")
print(f"  > Metadata: {job_result[0].metadata}")

#Sampler usage
sampler=AerSamplerV2()
num_shots=1024
print(f"\n### Sampler Usage: ###")

qc2 =my_circuit()
qc2.measure_all() ## add a measure all layer
# One pub, with one circuit to run against five different observables.
job = sampler.run([(qc2)], shots=num_shots)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]

print(f"The primitive-job finished with result {pub_result}")

counts = (
    pub_result.data.meas.get_int_counts()
)
dist = QuasiDistribution(
    {meas: count / num_shots for meas, count in counts.items()}, shots=num_shots
)

print(counts)
print(dist)


### Estimator Usage: ###
The primitive-job finished with result PubResult(data=DataBin<2>(evs=[ 0.07704939 -0.11584384], stds=[0.1 0.1]), metadata={'target_precision': 0.1, 'simulator_metadata': {'time_taken_parameter_binding': 7.205e-06, 'time_taken_execute': 8.8778e-05, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 15588, 'parallel_experiments': 1}})
  > Observables: ['ZI', 'IZ']
  > Expectation value: [ 0.07704939 -0.11584384]
  > Standard deviation: [0.1 0.1]
  > Metadata: {'target_precision': 0.1, 'simulator_metadata': {'time_taken_parameter_binding': 7.205e-06, 'time_taken_execute': 8.8778e-05, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 15588, 'parallel_experiments': 1}}

### Sampler Usage: ###
The primitive-job finished with result PubResult(data=DataBin<>(meas=BitArray(<shape=(), num_shots=1024, num_bits=2>)), metadata={'shots': 1024})
{0: 487, 3: 537}
{0: 0.4755859375, 3: 0.5244140625}


In [7]:
#Aer simulation noisy
from qiskit_ibm_runtime.fake_provider import FakeHanoi
from qiskit_aer.primitives import EstimatorV2 as AerEstimatorV2
from qiskit_aer.primitives import SamplerV2 as AerSamplerV2
from qiskit_aer.noise import NoiseModel

fake_backend=FakeHanoi()
coupling_map = fake_backend.configuration().coupling_map
noise_model = NoiseModel.from_backend(fake_backend)

print(f"\n### Estimator Usage: ###")

estimator=AerEstimatorV2(
    options={
          "backend_options":{

              "coupling_map": coupling_map,
              "noise_model": noise_model,
              "device": None #device
          }#,
    }
)
precision=0.1 #None #or between 0.0 and 1.0
qc=my_circuit()
# Estimator Usage
# One pub, with one circuit to run against five different observables.
job = estimator.run([(qc, observables)], precision=precision)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]
print(f"The primitive-job finished with result {pub_result}")
print(f"  > Observables: {observables_labels}")
print(f"  > Expectation value: {job_result[0].data.evs}")
print(f"  > Standard deviation: {job_result[0].data.stds}")
print(f"  > Metadata: {job_result[0].metadata}")

#Sampler usage
sampler=AerSamplerV2(
    options={
          "backend_options":{

              "coupling_map": coupling_map,
              "noise_model": noise_model,
              "device": None #device
          }#,
    }
)
num_shots=1024
print(f"\n### Sampler Usage: ###")

qc2 =my_circuit()
qc2.measure_all() ## add a measure all layer
# One pub, with one circuit to run against five different observables.
job = sampler.run([(qc2)], shots=num_shots)

# This is the result of the entire submission.  We submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had five observables,
# so contains information on all five.
pub_result = job.result()[0]

print(f"The primitive-job finished with result {pub_result}")

counts = (
    pub_result.data.meas.get_int_counts()
)
dist = QuasiDistribution(
    {meas: count / num_shots for meas, count in counts.items()}, shots=num_shots
)

print(counts)
print(dist)


### Estimator Usage: ###
The primitive-job finished with result PubResult(data=DataBin<2>(evs=[-0.07412772  0.12620763], stds=[0.1 0.1]), metadata={'target_precision': 0.1, 'simulator_metadata': {'time_taken_parameter_binding': 1.7911e-05, 'time_taken_execute': 0.012854319, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 15588, 'parallel_experiments': 1}})
  > Observables: ['ZI', 'IZ']
  > Expectation value: [-0.07412772  0.12620763]
  > Standard deviation: [0.1 0.1]
  > Metadata: {'target_precision': 0.1, 'simulator_metadata': {'time_taken_parameter_binding': 1.7911e-05, 'time_taken_execute': 0.012854319, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 15588, 'parallel_experiments': 1}}

### Sampler Usage: ###
The primitive-job finished with result PubResult(data=DataBin<>(meas=BitArray(<shape=(), num_shots=1024, num_bits=2>)), metadata={'shots': 1024})
{1: 10, 3: 522, 2: 11, 0: 481}
{1: 0.009765625, 3: 0.509765625, 2: 0.0107421875, 0: 0.4697265625}
